In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
raw_data_path=os.path.join('data','raw')
df=pd.read_csv(os.path.join(raw_data_path,'fer2013.csv'))
X=[]
for i in range(len(df)):
    X.append(df.loc[i,'pixels'].split(' '))
X=np.array(X,np.float32)/255.
y=df.emotion.values
X_train=X[np.where(df.Usage=='Training')]
y_train=y[np.where((df.Usage=='Training'))]
X_test=X[np.where((df.Usage!='Training'))]
y_test=y[np.where((df.Usage!='Training'))]
def balance_classes(X,y):
    X1=X[np.where(y!=1)]
    X2=X[np.where(y==1)]
    X2=np.repeat(X2,9,axis=0)
    X=np.concatenate((X1,X2),axis=0)
    y1=y[np.where(y!=1)]
    y2=y[np.where(y==1)]
    y2=np.repeat(y2,9,axis=0)
    y=np.concatenate((y1,y2),axis=0)
    i=[i for i in range(len(y))]
    np.random.shuffle(i)
    return X[i],y[i]
X_train,y_train=balance_classes(X_train,y_train)
def one_hot_encoder(X):
    N=len(X)
    K=len(set(X))
    Z=np.zeros((N,K))
    for i in range(N):
        c=X[i]
        Z[i,c]=1
    return Z

In [2]:
import torch as t

In [4]:
set(y_train)

{0, 1, 2, 3, 4, 5, 6}

In [6]:
device = t.device("cuda:0" if t.cuda.is_available() else "cpu")

In [27]:
len(X_test)

7178

In [72]:
class LeNet(t.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=t.nn.Conv2d(1,20,5,1)
        self.conv2=t.nn.Conv2d(20,50,5,1)   
        self.fc1=t.nn.Linear(9*9*50,500)
        self.fc2=t.nn.Linear(500,7)
    def forward(self,x):
        x=t.nn.functional.leaky_relu(self.conv1(x))
        x=t.nn.functional.max_pool2d(x,2,2)
        x=t.nn.functional.relu(self.conv2(x))
        x=t.nn.functional.max_pool2d(x,2,2)
        x=x.view(-1,9*9*50)
        x=t.nn.functional.relu(self.fc1(x))
        x=self.fc2(x)
        return x
    

In [73]:
model = LeNet().to(device)
model

LeNet(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=4050, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=7, bias=True)
)

In [74]:
criterion = t.nn.CrossEntropyLoss()
optimizer = t.optim.Adam(model.parameters(), lr = 0.00001)

In [75]:
inputs=X_train.reshape(32197,1,48,48)
labels=y_train

In [76]:
val_inputs_X=X_test.reshape(7178,1,48,48)
val_labels_y=y_test

In [77]:
inputs[1].shape


(1, 48, 48)

In [78]:
n_batches=320
batch_sz=100

In [79]:
epochs = 100
running_loss_history = []
running_corrects_history = []
val_running_loss_history = []
val_running_corrects_history = []
for e in range(epochs):
    running_loss = 0.0
    running_corrects = 0.0
    val_running_loss = 0.0
    val_running_corrects = 0.0
    for j in range(n_batches):
        Xbatch = inputs[j*batch_sz:(j*batch_sz+batch_sz)]
        Ybatch = labels[j*batch_sz:(j*batch_sz+batch_sz)]
        inputs_b = t.tensor(Xbatch).to(device)
        labels_b = t.tensor(Ybatch).to(device)
        outputs = model(inputs_b)
        loss = criterion(outputs, labels_b)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, preds = t.max(outputs, 1)
        running_loss += loss.item()
        running_corrects += t.sum(preds == labels_b.data)
    with t.no_grad():
        val_inputs = t.tensor(val_inputs_X).to(device)
        val_labels = t.tensor(val_labels_y).to(device)
        val_outputs = model(val_inputs)
        val_loss = criterion(val_outputs, val_labels)
        _, val_preds = t.max(val_outputs, 1)
        val_running_loss += val_loss.item()
        val_running_corrects += t.sum(val_preds == val_labels.data)
    epoch_loss = running_loss/len(training_loader)
    epoch_acc = running_corrects.float()/ len(training_loader)
    running_loss_history.append(epoch_loss)
    running_corrects_history.append(epoch_acc)
    val_epoch_loss = val_running_loss/len(validation_loader)
    val_epoch_acc = val_running_corrects.float()/ len(validation_loader)
    val_running_loss_history.append(val_epoch_loss)
    val_running_corrects_history.append(val_epoch_acc)
    print('epoch :', (e+1))
    print('training loss: {:.4f}, acc {:.4f} '.format(epoch_loss, epoch_acc.item()))
    print('validation loss: {:.4f}, validation acc {:.4f} '.format(val_epoch_loss, val_epoch_acc.item()))

epoch : 1
training loss: 1.0202, acc 11.8117 
validation loss: 0.0189, validation acc 17.9800 
epoch : 2
training loss: 1.0026, acc 12.3900 
validation loss: 0.0185, validation acc 19.2300 
epoch : 3
training loss: 0.9842, acc 13.4283 
validation loss: 0.0182, validation acc 21.6600 
epoch : 4
training loss: 0.9673, acc 15.1633 
validation loss: 0.0179, validation acc 23.4300 
epoch : 5
training loss: 0.9505, acc 16.5933 
validation loss: 0.0176, validation acc 24.5600 
epoch : 6
training loss: 0.9357, acc 17.5367 
validation loss: 0.0174, validation acc 25.4200 
epoch : 7
training loss: 0.9239, acc 18.1833 
validation loss: 0.0172, validation acc 25.9500 
epoch : 8
training loss: 0.9146, acc 18.6567 
validation loss: 0.0171, validation acc 26.1800 
epoch : 9
training loss: 0.9074, acc 19.0450 
validation loss: 0.0169, validation acc 26.4800 
epoch : 10
training loss: 0.9015, acc 19.3133 
validation loss: 0.0169, validation acc 26.6300 
epoch : 11
training loss: 0.8965, acc 19.5317 
va

training loss: 0.6640, acc 28.9183 
validation loss: 0.0140, validation acc 33.5500 
epoch : 87
training loss: 0.6617, acc 29.0150 
validation loss: 0.0140, validation acc 33.6300 
epoch : 88
training loss: 0.6595, acc 29.0867 
validation loss: 0.0139, validation acc 33.7000 
epoch : 89
training loss: 0.6573, acc 29.1600 
validation loss: 0.0139, validation acc 33.7000 
epoch : 90
training loss: 0.6551, acc 29.2267 
validation loss: 0.0139, validation acc 33.7500 
epoch : 91
training loss: 0.6530, acc 29.3217 
validation loss: 0.0139, validation acc 33.8100 
epoch : 92
training loss: 0.6508, acc 29.4067 
validation loss: 0.0138, validation acc 33.9100 
epoch : 93
training loss: 0.6487, acc 29.5083 
validation loss: 0.0138, validation acc 33.9800 
epoch : 94
training loss: 0.6466, acc 29.5800 
validation loss: 0.0138, validation acc 34.0500 
epoch : 95
training loss: 0.6445, acc 29.6367 
validation loss: 0.0138, validation acc 34.1500 
epoch : 96
training loss: 0.6425, acc 29.7333 
vali